In [1]:
import os
import pandas as pd

In [2]:
##### INPUTS #####
dataset_name = "race_pp_4000"
seed = 123
model_name = "transformer"

In [9]:
def get_metrics(dataset_name: str, model_name: str, seed: int) -> pd.DataFrame:
    """Read metrics from output directory and return as a dataframe."""
    output_dir = os.path.join('output', dataset_name, 'seed_' + str(seed))

    metrics = pd.read_csv(os.path.join(output_dir, 'eval_metrics_' + model_name + '.csv'))
    metrics = metrics.transpose().reset_index().rename(columns={'index': 'metric', 0: 'value'})
    return metrics

metrics = get_metrics(dataset_name, seed, model_name)
metrics

,metric,value
0,test_mean_absolute_error,0.216413
1,train_mean_absolute_error,0.160750
2,test_root_mean_squared_error,0.470882
3,train_root_mean_squared_error,0.412816
4,test_r2_score,0.389775
5,train_r2_score,0.744375
6,test_spearman_rho,0.717989
7,train_spearman_rho,0.873924
8,test_pearson_rho,0.730308
9,train_pearson_rho,0.873902
